<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_4_OLCI_ope_rep_time_series.ipynb" target="_blank"><< Acquiring OLCI timeseries from operational and reprocessed collections</a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn OLCI: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Users should review the following notebooks for more information on setting up a credentials files for using the eumdac python library to retrieve OLCI data from the Data Store:
    
- **<a href="../1_OLCI_introductory/1_1a_OLCI_data_access_Data_Store.ipynb">1_1a_OLCI_data_access_Data_Store.ipynb</a>**

You will also need to have registered for a **<a href="https://eoportal.eumetsat.int/" target="_blank">A EUMETSAT Earth Observation Portal account</a>** to download from the EUMETSAT Data Store.

</div>
<hr>

# 2.5 OLCI advanced chlorophyll comparisons

### Data used

| Product Description | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-------------:|
| Sentinel-3 OLCI level-2 reduced resolution | EO:EUM:DAT:0408 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_2_WRR___NTC?query=reduced%20resolution&&results=24&s=advanced" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* 

### Outline


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Step 1: Acquiring OLCI data](#section1)

<hr>

When we use Python, we nearly always start by importing the libraries we need. Each library gives us access to the capabilities we need to perform specific tasks. We can import library as shown in the box below.

In [1]:
import datetime     # a libary that allows us to work with dates and times
import json         # a library that helps us make JSON format files
import numpy as np  # a library that lets us work with arrays; we import this with a new name "np"
import os           # a library that allows us access to basic operating system commands like making directories
from shapely import geometry # a library that support construction of geometry objects
import shutil       # a library that allows us access to basic operating system commands like copy
import xml.etree.ElementTree as ET # a library that allows us to work with XML files
import zipfile      # a library that allows us to unzip zip-files.
import eumdac       # a tool that helps us download via the eumetsat/data-store

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Step 1: Acquiring OLCI data
[Back to top](#TOC_TOP)

</div>

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More **essential** information on setting this up can be found in the **<a href="../1_OLCI_introductory/1_1a_OLCI_data_access_Data_Store.ipynb">1_1a_OLCI_data_access_Data_Store</a>** notebook.

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token 'f5e2024e-e500-371b-969c-1745a2abe1b4' expires 2022-08-31 11:24:56.938965


We will work with the OLCI Level-2 reduced resolution data throughout this notebook. The **collection ID** for this data type is `EO:EUM:DAT:0408`. You can find this information on the Data Store (https://data.eumetsat.int/), or ask the `eumdac` client to tell you all the avaiable collections by calling the `eumdac.DataStore(token).collections` method. We can use our collection ID to create a collection object (`selected_collection`) and ask for more information on the collection as follows.

In [4]:
# set collection ID for OLCI L2 EFR
collectionID = 'EO:EUM:DAT:0408'

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(f"{selected_collection.title}\n---\n{selected_collection.abstract}")

OLCI Level 2 Ocean Colour Reduced Resolution - Sentinel-3
---
OLCI (Ocean and Land Colour Instrument) Ocean Colour Geophysical Products. Reduced Resolution: 1200m at nadir. Level 2 marine products include the following: * water-leaving reflectances in 16 bands, Oa**_reflectance (Baseline Atmospheric Correction (BAC) algorithm, bands: 400, 412, 442, 490, 510, 560, 620, 665, 674, 681, 709, 754, 779, 865, 885, 1024 nanometer (nm)); *algal pigment concentration in clear waters, chl_oc4me (BAC and maximum band ratio algorithm, log10 scaled); *algal pigment concentration in turbid waters, chl_nn (neural net algorithm, log10 scaled); *total suspended matter concentration, tsm_nn (neural net algorithm, log10 scaled); *diffuse attenuation coefficient Kd of downward irradiance at 490 nm, trsp (BAC and M07 algorithm, log10 scaled); *absorption coefficient at 443 nm of coloured detrital and dissolved organic matter, iop_nn (neural net algorithm, log10 scaled); *instantaneous photosynthetically act

Now we have defined our collection, we can request products from it. In this case we are interested in the specific product defined below.

In [5]:
productID = 'S3B_OL_2_WRR____20220318T071751_20220318T080148_20220319T115438_2637_063_377______MAR_O_NT_003.SEN3'
selected_product = datastore.get_product(product_id=productID, collection_id=collectionID)

Now we will download the product (which will be zipped), unzip it, and clear up the latent zip file.

In [6]:
# Download the product
with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
    print(f'Downloading {fsrc.name}')
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {fsrc.name} finished.')

# Unzip the product
with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith(str(selected_product)):
            zip_ref.extract(file, download_dir)
    print(f'Unzipping of product {selected_product} finished.')

# Clean up
os.remove(fdst.name)

Download of product S3B_OL_2_WRR____20220318T071751_20220318T080148_20220319T115438_2637_063_377______MAR_O_NT_003.SEN3.zip finished.
Unzipping of product S3B_OL_2_WRR____20220318T071751_20220318T080148_20220319T115438_2637_063_377______MAR_O_NT_003.SEN3 finished.


If all went well, your products are now downloaded. You can find them in the `products` directory in the same folder as this notebook.

<div class="alert alert-block alert-warning">

### Challenge:

Now you have run this for acquiring the OLCI L1 full resolution (EFR) data, can you do adapt the script above to get the corresponding granules for the OLCI L2 full resolution (WFR) granules?

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_4_OLCI_ope_rep_time_series.ipynb" target="_blank"><< Acquiring OLCI timeseries from operational and reprocessed collections</a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>